In [1]:
import cv2
import numpy as np

# Initialize camera and ORB detector
cap = cv2.VideoCapture(0)
detector = cv2.ORB_create()

# Initialize empty lists for keypoints and descriptors
kp_list = []
des_list = []

while True:
    # Capture frame from camera
    ret, frame = cap.read()
    
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect keypoints and descriptors using ORB
    keypoints, descriptors = detector.detectAndCompute(gray, None)
    
    # Add keypoints and descriptors to lists
    kp_list.append(keypoints)
    des_list.append(descriptors)
    
    # If we have more than one frame, match keypoints and estimate motion
    if len(kp_list) > 1:
        # Match keypoints between last two frames
        matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        matches = matcher.match(des_list[-2], des_list[-1])
        
        # Filter out bad matches using RANSAC
        src_pts = np.float32([kp_list[-2][m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp_list[-1][m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        
        # Draw matches on current frame
        match_img = cv2.drawMatches(frame, kp_list[-2], frame, kp_list[-1], matches, None, flags=2)
        
        # If we have a good homography, update our position estimate
        if M is not None:
            x, y, z = M[:3, 2]

    # Display the current frame
        cv2.imshow("SLAM", match_img)
        key = cv2.waitKey(1)
        if key == ord('q'):
            break

# Release the camera and close the window
cap.release()
cv2.destroyAllWindows()
